# # Question1

In [ ]:
import pandas as pd

#Read GDP per capita and metadata csv files
gdp_data = pd.read_csv('API_NY.GDP.PCAP.CD_DS2_en_csv_v2_5728786.csv')
gdp_metadata = pd.read_csv('Metadata_Country_API_NY.GDP.PCAP.CD_DS2_en_csv_v2_5728786.csv')

# Read malnutrition and metadata csv files
malnutrition_data = pd.read_csv('API_SH.STA.MALN.ZS_DS2_en_csv_v2_5736088.csv')
malnutrition_metadata = pd.read_csv('Metadata_Country_API_SH.STA.MALN.ZS_DS2_en_csv_v2_5736088.csv')

# Clean and reshape GDP_per_capita data 
gdp_data = gdp_data.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='GDP_per_capita')

# Clean and reshape malnutrition data
malnutrition_data = malnutrition_data.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='Malnutrition_prevalence')

# Merge GDP per capita and malnutrition data
gdp_malnutrition = pd.merge(gdp_data, malnutrition_data, on=['Country Code', 'Year'])

import matplotlib.pyplot as plt

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(gdp_malnutrition['GDP_per_capita'], gdp_malnutrition['Malnutrition_prevalence'], alpha=0.6)
plt.xlabel('GDP per Capita (current US$)')
plt.ylabel('Malnutrition Prevalence (% of children under 5)')
plt.title('Relationship Between GDP per Capita and Malnutrition Prevalence')
plt.grid(True)
plt.show()

# Regional analysis 

In [ ]:
# Merge, metadata with gdp_with_malnutrition
gdp_malnutrition_with_metadata = pd.merge(gdp_malnutrition, gdp_metadata[['Country Code', 'Region']], on='Country Code')

import seaborn as sns

# Exclude countries in North America
non_north_america_data = gdp_malnutrition_with_metadata[gdp_malnutrition_with_metadata['Region'] != 'North America']

# Create a scatter plot with colored regions
plt.figure(figsize=(10, 6))
sns.scatterplot(data=non_north_america_data, x='GDP_per_capita', y='Malnutrition_prevalence', hue='Region', alpha=0.6)
plt.xlabel('GDP per Capita (current US$)')
plt.ylabel('Malnutrition Prevalence (% of children under 5)')
plt.title('Relationship Between GDP per Capita and Malnutrition Prevalence by regions (Excluding North America)')
plt.grid(True)
plt.legend(title='Region')
plt.show()

# Income level analysis

In [ ]:
gdp_malnutrition_with_metadata_income = pd.merge(gdp_malnutrition, gdp_metadata[['Country Code', 'IncomeGroup']], on='Country Code')

# Create a scatter plot with colored income groups
plt.figure(figsize=(10, 6))
sns.scatterplot(data=gdp_malnutrition_with_metadata_income, x='GDP_per_capita', y='Malnutrition_prevalence', hue='IncomeGroup', alpha=0.6)
plt.xlabel('GDP per Capita (current US$)')
plt.ylabel('Malnutrition Prevalence (% of children under 5)')
plt.title('Relationship Between GDP per Capita and Malnutrition Prevalence by Income Level (Excluding North America)')
plt.grid(True)
plt.legend(title='Income Group')
plt.show()

# # Question2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import quandl

# Set your Quandl API key
quandl.ApiConfig.api_key = 'CWxMkUxtA-fyMF9hCgnT'

# Quandl codes for the datasets
wheat_code = 'ODA/PWHEAMT_USD'
oil_code = 'WGEC/WLD_CRUDE_WTI'
gold_code = 'BUNDESBANK/BBK01_WT5511'

# Download data from Quandl and set the index without specifying a column name
wheat_data = quandl.get(wheat_code)
oil_data = quandl.get(oil_code)
gold_data = quandl.get(gold_code)

# Resample data to daily frequency
wheat_data = wheat_data.resample('D').interpolate(method='linear')
oil_data = oil_data.resample('D').interpolate(method='linear')
gold_data = gold_data.resample('D').interpolate(method='linear')

# Merge the DataFrames on the index
merged_data = pd.concat([wheat_data, oil_data, gold_data], axis=1)
merged_data.columns = ['Wheat Prices', 'Crude Oil Prices', 'Gold Prices']

# Find maximum and minimum prices and their corresponding dates
wheat_max_date = merged_data['Wheat Prices'].idxmax()
wheat_max_price = merged_data['Wheat Prices'].max()
wheat_min_date = merged_data['Wheat Prices'].idxmin()
wheat_min_price = merged_data['Wheat Prices'].min()

oil_max_date = merged_data['Crude Oil Prices'].idxmax()
oil_max_price = merged_data['Crude Oil Prices'].max()
oil_min_date = merged_data['Crude Oil Prices'].idxmin()
oil_min_price = merged_data['Crude Oil Prices'].min()

gold_max_date = merged_data['Gold Prices'].idxmax()
gold_max_price = merged_data['Gold Prices'].max()
gold_min_date = merged_data['Gold Prices'].idxmin()
gold_min_price = merged_data['Gold Prices'].min()

# Plot the synchronized data on the same graph
plt.figure(figsize=(12, 6))
plt.plot(merged_data.index, merged_data['Wheat Prices'], label='Wheat Prices')
plt.plot(merged_data.index, merged_data['Crude Oil Prices'], label='Crude Oil Prices')
plt.plot(merged_data.index, merged_data['Gold Prices'], label='Gold Prices')

# indicate maximum and minimum prices with colored dots and legends
plt.scatter(wheat_max_date, wheat_max_price, color='green', marker='o', label=' Max Price')
plt.scatter(wheat_min_date, wheat_min_price, color='red', marker='o', label='Min Price')

plt.scatter(oil_max_date, oil_max_price, color='green', marker='o')
plt.scatter(oil_min_date, oil_min_price, color='red', marker='o')

plt.scatter(gold_max_date, gold_max_price, color='green', marker='o')
plt.scatter(gold_min_date, gold_min_price, color='red', marker='o')

plt.ylabel('Prices in $')
plt.xlabel('Date')
plt.title('Commodity Prices Over Time')
plt.legend()
plt.grid(True)
plt.show()



# # Question3

In [ ]:
import pandas
from pandas_datareader import wb
import matplotlib.pyplot as plt

#retrieve data from wb
Co2_emissions_per_metric_tones = 'EN.ATM.CO2E.PC'
school_enrolment_primary = 'SE.PRM.NENR'

#dowload all country data for the year 2010
df1 = wb.download(indicator = Co2_emissions_per_metric_tones, country='all', start=2010,end=2010)
df2 = wb.download(indicator = school_enrolment_primary, country='all', start=2010,end=2010)

# Calculate custom percentiles
custom_percentiles = [0.05, 0.25, 0.5, 0.75, 0.95] # 0.5(median)

# compute summary statis
summary1_stati = df1.describe(percentiles=custom_percentiles)
summary1_stati = summary1_stati.rename(columns={'50%': 'median'})
summary2_stati = df2.describe(percentiles=custom_percentiles)
summary2_stati = summary2_stati.rename(columns={'50%': 'median'})

print("Table showing Summary Statistics for CO2 Emissions (metric tons per capita)for all countries, year 2010")
print('\n', summary1_stati)
print("\nTable showing Summary Statistics for Primary School Enrolment (% net) for all countries, year 2010")
print('\n', summary2_stati)

# # Question4

In [ ]:
import pandas
from pandas_datareader import wb
import matplotlib.pyplot as plt
import numpy as np
#Get data from wb
Fertility_rate_birth_per_woman = 'SP.DYN.TFRT.IN'
GDP_per_capita = 'NY.GDP.PCAP.CD'

df1 = wb.download(indicator = Fertility_rate_birth_per_woman, country=[], start=2010,end=2010)
df2 = wb.download(indicator = GDP_per_capita, country=[], start=2010,end=2010)
df = df1.merge(df2, on="country")
df.columns = ["Fertility_rate_total(births_per_woman)", "GDP_per_capita(current US$)"]


plt.figure(figsize=(16, 6))
plt.scatter(df["GDP_per_capita(current US$)"], df["Fertility_rate_total(births_per_woman)"], alpha=0.8)
plt.title("Fertility Rate vs GDP per Capita (2010)")
plt.xlabel("GDP per Capita (current US$)")
plt.ylabel("Fertility Rate (births per woman)")
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Get data from World Bank
df1990 = wb.download(indicator=Fertility_rate_birth_per_woman, country=[], start=1990, end=1990)
df2010 = wb.download(indicator=Fertility_rate_birth_per_woman, country=[], start=2010, end=2010)

# Remove missing values
df1990 = df1990.dropna()
df22010 = df2010.dropna()

# Extract fertility rate data for both years
fertility_rate_1990 = df1990[Fertility_rate_birth_per_woman].dropna()
fertility_rate_2010 = df2010[Fertility_rate_birth_per_woman].dropna()

# Sort the data in ascending order for both years
sorted_data_1990 = np.sort(fertility_rate_1990)
sorted_data_2010 = np.sort(fertility_rate_2010)

# Calculate the CDF values for both years
cdf_1990 = np.arange(1, len(sorted_data_1990) + 1) / len(sorted_data_1990)
cdf_2010 = np.arange(1, len(sorted_data_2010) + 1) / len(sorted_data_2010)

# Calculate mean and median for 1990
mean_1990 = df1990[Fertility_rate_birth_per_woman].mean()
median_1990 = df1990[Fertility_rate_birth_per_woman].median()

# Calculate mean and median for 2010
mean_2010 = df2010[Fertility_rate_birth_per_woman].mean()
median_2010 = df2010[Fertility_rate_birth_per_woman].median()

# CDF plots for both years
plt.figure(figsize=(12, 6))

# CDF plot for 1990
plt.subplot(1, 2, 1)
plt.plot(sorted_data_1990, cdf_1990, label='CDF (1990)')
plt.xlabel('Fertility Rate')
plt.ylabel('Cumulative Probability')
plt.title('Fertility Rate CDF (1990)')
plt.axvline(mean_1990, color='r', linestyle='--', label='Mean (1990)')
plt.axvline(median_1990, color='g', linestyle='--', label='Median (1990)')
plt.legend()

# CDF plot for 2010
plt.subplot(1, 2, 2)
plt.plot(sorted_data_2010, cdf_2010, label='CDF (2010)')
plt.xlabel('Fertility Rate')
plt.ylabel('Cumulative Probability')
plt.title('Fertility Rate CDF (2010)')
plt.axvline(mean_2010, color='r', linestyle='--', label='Mean (2010)')
plt.axvline(median_2010, color='g', linestyle='--', label='Median (2010)')
plt.legend()

plt.tight_layout()
plt.show()

# # Question5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the HPI and CPI datasets from Excel files
hpi_data = pd.read_excel('hpi-data-2016.xlsx')
cpi_data = pd.read_excel('CPI2016-Results.xlsx')

# Merge datasets based on matching countries
merged_data = pd.merge(hpi_data, cpi_data, on='Country', how='inner')

# Calculate ranks for HPI and CPI
merged_data['HPI'] = merged_data['HPI Rank'].rank(ascending=False)
merged_data['CPI'] = merged_data['CPI rank'].rank(ascending=True)

# Create a scatter plot using ranked values
plt.figure(figsize=(12, 6))
plt.scatter(merged_data['CPI'], merged_data['HPI'], alpha=0.6)
plt.title(" HPI vs. CPI (Ranks) year 2016")
plt.xlabel("Corruption Perceptions Index (CPI) Rank")
plt.ylabel("Happy Planet Index (HPI) Rank")

#annotate countries
for idx, row in merged_data.iterrows():
    plt.annotate(row['Country'], (row['CPI'], row['HPI']), fontsize=8)

# Annotate unusual countries (customize as needed)
unusual_countries = merged_data[(merged_data['HPI'] > 110) & (merged_data['CPI'] > 90)]
for index, row in unusual_countries.iterrows():
    plt.annotate(row['Country'], (row['CPI'], row['HPI']), fontsize=8, color='red')

# Show the plot
plt.grid(True)
plt.tight_layout()
plt.show()